# Julia Code
from https://juliadynamics.github.io/DynamicalSystems.jl/latest/chaos/lyapunovs/#maximum-lyapunov-exponent

In [1]:
using Pkg; Pkg.update("DynamicalSystems")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


In [2]:
using Pkg;
Pkg.update()

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Installed MbedTLS_jll ────────── v2.16.8+1
  Installed LaTeXStrings ───────── v1.2.1
  Installed Formatting ─────────── v0.4.2
  Installed FFMPEG ─────────────── v0.4.0
  Installed FillArrays ─────────── v0.8.14
  Installed IJulia ─────────────── v1.23.2
  Installed ForwardDiff ────────── v0.10.18
  Installed Plots ──────────────── v1.6.12
  Installed PlotThemes ─────────── v2.0.1
  Installed RecipesPipeline ────── v0.1.13
  Installed CommonSubexpressions ─ v0.3.0
  Installed Latexify ───────────── v0.14.12
  Installed GeometryBasics ─────── v0.3.12
  Installed Distributions ──────── v0.23.8
  Installed StructArrays ───────── v0.5.1
  Installed ArrayInterface ─────── v3.1.1
  Installed GR ─────────────────── v0.52.0
  Installed ColorSchemes ───────── v3.12.1
  Installed PlotUtils ──────────── v1.0.10
  Installed RecipesBase ────────── v1.1.1
  Installed MbedTLS ────────────── v1.0.3
  Installed EarCut_jll ─────────── v2.1.5+1
##########################################################

In [ ]:
using DynamicalSystems, PyPlot

┌ Info: Precompiling DynamicalSystems [61744808-ddfa-5f27-97ff-6e42cc95d634]
└ @ Base loading.jl:1260


In [ ]:
henon = Systems.henon()
tr1 = trajectory(henon, 100)
summary(tr1)

In [ ]:
u2 = get_state(henon) + (1e-9 * ones(2))
tr2 = trajectory(henon, 100, u2)
summary(tr2)

In [ ]:
using LinearAlgebra: norm

fig = figure()

# Plot the x-coordinate of the two trajectories:
ax1 = subplot(2,1,1)
plot(tr1[:, 1], alpha = 0.5)
plot(tr2[:, 1], alpha = 0.5)
ylabel("x")

# Plot their distance in a semilog plot:
ax2 = subplot(2,1,2, sharex = ax1)
d = [norm(tr1[i] - tr2[i]) for i in 1:length(tr2)]
ylabel("d"); xlabel("n"); semilogy(d);
fig.tight_layout(pad=0.3); 

The initial slope of the d vs n plot (before the curve saturates) is approximately the maximum Lyapunov exponent!

# EX 2

In [ ]:
using DynamicalSystems
ds = Systems.towel()
λλ = lyapunovs(ds, 10000)

In [ ]:
lor = Systems.lorenz(ρ = 32.0) #this is not the original parameter!
λλ = lyapunovs(lor, 10000, dt = 0.1)

In [ ]:
lor

In [ ]:
using DynamicalSystems, PyPlot

he = Systems.henon()
as = 0.8:0.005:1.225; λs = zeros(length(as), 2)
for (i, a) in enumerate(as)
    set_parameter!(he, 1, a)
    λs[i, :] .= lyapunovs(he, 10000; Ttr = 500)
end

fig = figure()
plot(as, λs); xlabel("\$a\$"); ylabel("\$\\lambda\$")
fig.tight_layout(pad=0.3); 

# Defining new dynamical systems

In [ ]:
using DynamicalSystems # also exports relevant StaticArrays names
# Lorenz system
# Equations of motion:
@inline @inbounds function loop(u, p, t)
    σ = p[1]; ρ = p[2]; β = p[3]
    du1 = σ*(u[2]-u[1])
    du2 = u[1]*(ρ-u[3]) - u[2]
    du3 = u[1]*u[2] - β*u[3]
    return SVector{3}(du1, du2, du3)
end
# Jacobian:
@inline @inbounds function loop_jac(u, p, t)
    σ, ρ, β = p
    J = @SMatrix [-σ  σ  0;
    ρ - u[3]  (-1)  (-u[1]);
    u[2]   u[1]  -β]
    return J
end

ds = ContinuousDynamicalSystem(loop, [0.0, 10.0, 0.0], [10.0, 32.0, 8/3], loop_jac)

In [ ]:
lyapunovs(ds, 10000, dt = 0.1)

In [ ]:
# Henon map.
# equations of motion:
function hiip(dx, x, p, n)
    dx[1] = 1.0 - p[1]*x[1]^2 + x[2]
    dx[2] = p[2]*x[1]
    return
end
# Jacobian:
function hiip_jac(J, x, p, n)
    J[1,1] = -2*p[1]*x[1]
    J[1,2] = 1.0
    J[2,1] = p[2]
    J[2,2] = 0.0
    return
end
ds = DiscreteDynamicalSystem(hiip, zeros(2), [1.4, 0.3], hiip_jac)

In [ ]:
lyapunovs(ds, 10000, dt = 0.1)